In [3]:
from code.monte_carlo import monte_carlo
from code.finmath1 import BlackScholes

In [5]:
# Arguments

S0 = 100
K = 100
T = 1.0
r = 0.05
sigma = 0.2
alpha = 0.8
N = 5_000_000

# Calculate prices

price_call = monte_carlo(S0, K, T, r, sigma, alpha, N, call=True)
price_put = monte_carlo(S0, K, T, r, sigma, alpha, N, call=False)
print(f'(CALL) Monte-Carlo method (alpha = {alpha}) price = {price_call:.4f}')
print(f'(PUT) Monte-Carlo method (alpha = {alpha}) price = {price_put:.4f}\n')

alpha_classic = 1.0
price_classic_call = monte_carlo(S0, K, T, r, sigma, alpha_classic, N, call=True)
price_classic_put = monte_carlo(S0, K, T, r, sigma, alpha_classic, N, call=False)
print(f'(CALL) Monte-Carlo method (alpha = {alpha_classic}) price = {price_classic_call:.4f}')
print(f'(PUT) Monte-Carlo method (alpha = {alpha_classic}) price = {price_classic_put:.4f}\n')

model = BlackScholes(sigma, r)
print(f'(CALL) Analytic solution (alpha = 1.0) price = {model.call_price(S0, T, K):.4f}')
print(f'(PUT) Analytic solution (alpha = 1.0) price = {model.put_price(S0, T, K):.4f}\n')

(CALL) Monte-Carlo method (alpha = 0.8) price = 10.6016
(PUT) Monte-Carlo method (alpha = 0.8) price = 5.4197

(CALL) Monte-Carlo method (alpha = 1.0) price = 10.4486
(PUT) Monte-Carlo method (alpha = 1.0) price = 5.5634

(CALL) Analytic solution (alpha = 1.0) price = 10.4506
(PUT) Analytic solution (alpha = 1.0) price = 5.5735

